# 1. Install and call packages

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats.distributions import chi2 
from arch import arch_model
from math import sqrt
import seaborn as sns
sns.set()
import scipy.optimize as sco
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf

# 2. Define GARCH function

In [2]:
def GARCH_predict(symbol_list, start, end, interval): 
    
    #download data and calculate returns
    data = yf.download(symbol_list, start, end, interval = interval)
    ret = data.pct_change()['Adj Close']
    ret = ret.dropna()
    
    #create list to store predicted variance and volatility
    variance_list = []
    vol_list = []
    
    for symbol in symbol_list:
        
        model = arch_model(ret[symbol], 
                            mean = "Constant",
                            vol = "GARCH", 
                            dist = 'normal', 
                            p = 1, q = 1, 
                            rescale = False) 
       
        result = model.fit(update_freq = 5, disp = 'off')
        forecast = result.forecast()
        
        predict_var = (forecast.variance.iloc[-1]).iloc[0]
        variance_list.append(predict_var)
        vol_list.append(np.sqrt(predict_var))
        
        # It's optional to print other statistical result
        # print(result.plot())
        # print(result.summary())
        # print(forecast.mean)

    df = pd.DataFrame(columns = symbol_list, index = ['predicted var','predicted vol'])
    df.loc['predicted var'] = variance_list
    df.loc['predicted vol'] = vol_list
    
    # The function returns a DataFrame containing predicted variance and volatility values.
    return(df)



# 3. Define Basic MVO Framework

In [3]:
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    
    # Recall portfolio_annualised_performance(weights, mean_returns, cov_matrix) returns portfolio standard deviation and portfolio return
    p_var = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    p_ret = np.sum(mean_returns*weights)
    return -(p_ret - risk_free_rate/52) / p_var

def max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0,0.25)
    bounds = tuple(bound for asset in range(num_assets))
    
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result

def MVO_result(df,mean_returns, cov_matrix, risk_free_rate):    

    max_sharpe = max_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
    print ("-"*80)
    print ("Maximum Sharpe Ratio Portfolio Allocation\n")
    print (max_sharpe)
    
    weights = max_sharpe['x']
    rp = np.sum(mean_returns*weights)
    sdp = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    
    max_sharpe_allocation = pd.DataFrame(max_sharpe.x,index=df.columns,columns=['allocation'])
    max_sharpe_allocation.allocation = [round(i*100,2)for i in max_sharpe_allocation.allocation]
    max_sharpe_allocation = max_sharpe_allocation.T
    
    print ("-"*80)
    print ("Weekly Return:", round(rp,5))
    print ("Weekly Volatility:", round(sdp,5))
    print ("Max Weekly Sharpe Ratio:", (rp - (risk_free_rate/52))/sdp)
    print ("\n")
    print (max_sharpe_allocation)
    return max_sharpe.x

# 4. Seclet Stocks Based on Valuation Matrix Score

The FactSet-sourced file contains valuation scores for SP500 composition stocks, categorized by industry. I re-grouped the data by narrowing industry types down to 9 types only: Financials, Chemicals, Tech, Utilities, Air, F&B, Oil, Services and Others. The criteria used in my trading is to select the stocks with best combined score in each industry

In [4]:
# upload stock score data scv file (downloaded from FactSet)
stock = pd.read_csv('./files/Scoring the SP 500 - Valuation and Sales Growth.csv', na_values=['#N/A'])

# set index by symbol
stock = stock.set_index('Symbol')

# look for the max score within each industry
stock['score_max'] = stock.groupby(['Industry'])['Combined Score'].transform(max)

# select stocks with industry max score 
selection = stock[stock['Combined Score']>=stock['score_max']*0.99]

In [6]:
selection

,Name,Industry,Combined Score,Sales Growth (1 Year),P/E,P/E Relative to Industry,Price/Book,Price/Book Relative to Industry,Price/Sales,Price/Sales Relative to Industry,score_max
Symbol,,,,,,,,,,,
NVDA,NVIDIA Corporation,Others,9.8,-6.8,55.4,1.4,12.6,1.8,14.2,2.3,9.8
HPE,Hewlett Packard Enterprise Co.,Tech,8.8,-6.4,21.4,1.4,1.2,1.0,0.8,1.3,8.8
SLB,Schlumberger NV,Oil,8.8,0.3,NaN,NaN,2.3,1.4,1.7,1.7,8.8
WEC,WEC Energy Group Inc,Utilities,8.6,-2.0,25.8,1.1,2.9,1.1,3.9,1.4,8.6
APD,"Air Products and Chemicals, Inc.",Chemicals,8.2,-0.1,27.9,1.0,4.4,1.2,5.5,1.9,8.2
ECL,Ecolab Inc.,Chemicals,8.2,1.6,36.2,1.3,6.4,1.7,3.8,1.3,8.2
HRL,Hormel Foods Corporation,F&B,8.2,-0.5,22.6,1.2,3.7,1.3,2.3,1.5,8.2
LUV,Southwest Airlines Co.,Air,8.2,2.1,12.6,1.4,2.8,1.2,1.3,1.7,8.2
UDR,"UDR, Inc.",Financials,8.2,-2.1,73.8,1.3,4.2,0.9,11.5,1.2,8.2


# 5. Call Functions to Calculate Allocation for chozen stocks

In [5]:
# input parameters
symbol_list = selection.index.tolist()
end = dt.datetime.now()
start = end - dt.timedelta(140)
interval = "1wk"

# download data
returns = yf.download(symbol_list, start, end, interval = interval).pct_change()['Adj Close'].dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()
risk_free_rate = 0.12 / 100

# print optimal allocation using historical covarianc matrix
allocation_hist = MVO_result(returns, mean_returns, cov_matrix, risk_free_rate)
print('MVO result by historical covariance matrix')
print(allocation_hist)

# replace diagonal elements of cov matrix by GARCH-predicted variance.
GARCH_var = GARCH_predict(symbol_list, start, end, interval)
adjust_cov_matrix = cov_matrix
for symbols in symbol_list:
    adjust_cov_matrix[symbols][symbols] = GARCH_var[symbols][0]

# print optimal allocation using GARCH covariance matrix
allocation_GARCH = MVO_result(returns, mean_returns, adjust_cov_matrix, risk_free_rate)
print('MVO result by GARCH-based covariance matrix')
print(allocation_GARCH)

# print the shrinkage allocation
print('-'*80)
print('MVO shrinkage result')
print(0.5*allocation_hist + 0.5*allocation_GARCH)

[*********************100%***********************]  11 of 11 completed
--------------------------------------------------------------------------------
Maximum Sharpe Ratio Portfolio Allocation

     fun: -0.5604631537588548
     jac: array([ 0.14930844,  0.77556829,  0.42017675,  0.14876426,  0.14896841,
       -0.37233631,  0.61472553,  1.10447762,  0.55416086,  0.06916744,
        0.20322725])
 message: 'Optimization terminated successfully.'
    nfev: 91
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.24251515, 0.        , 0.        , 0.19875002, 0.05873483,
       0.25      , 0.        , 0.        , 0.        , 0.25      ,
       0.        ])
--------------------------------------------------------------------------------
Weekly Return: 0.01024
Weekly Volatility: 0.01823
Max Weekly Sharpe Ratio: 0.5604631537588548


              APD  ECL  HPE    HRL   LUV  NVDA  OMC  SLB  UDR   WEC  WMT
allocation  24.25  0.0  0.0  19.88  5.87  25.0  0.0  0.0  0.0  25.0  0.0

# 4. Reference
* https://campus.datacamp.com/courses/garch-models-in-python/garch-model-fundamentals?ex=9
* https://stackoverflow.com/questions/59884917/forecasting-volatility-using-garch-in-python-arch-package
* https://stackoverflow.com/questions/15705630/get-the-rows-which-have-the-max-count-in-groups-using-groupby
* Professor Lee's BootCamp Videos